Integral approximation (p. 52, ex. 6.8). Test function with polynomials with power of 2 and 4, test on some random function.

For given $f(x)$ count $\int_{0}^{1} f(x) dx$ approximately by composite cubature rules:
- left rectangles
- trapezes
- simpson's

In [1]:
import numpy as np
import pandas as pd

from scipy import integrate

# Integral approximation formulas 

In [2]:
def left_rectangles(f, N, a=0, b=1):
    """Composite left triangles formula.
    
    Args:
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        a, b (int, int): Bounds.
        
    Returns:
        result (float): Approximate integral value.
    """
    
    # Length of part split.
    h = (b - a) / N
    
    # Points.
    x = np.array([a + (k - 1) * h for k in range(1, N + 1)])
    
    # Values in points.
    y = f(x)
    
    # Integral value.
    result = h * np.sum(y)

    return result    

In [3]:
def trapezes(f, N, a=0, b=1):
    """Composite trapezes formula.
    
    Args:
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        a, b (int, int): Bounds.
        
    Returns:
        result (float): Approximate integral value.
    """
    
    # Length of part split.
    h = (b - a) / N
    
    # Points.
    x = np.linspace(a, b, N + 1)
    
    # Values in points.
    y = f(x)  
        
    # Integral value.
    result = (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])
    
    return result    

In [4]:
def simpson(f, N, a=0, b=1):
    """Composite simpson formula.
    
    Args:
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        a, b (int, int): Bounds.
        
    Returns:
        result (float): Approximate integral value.
    """
    
    if N % 2 == 1:
        raise ValueError("N must be an even integer.")
    
    # Length of part split.
    h = (b - a) / N
    
    # Points.
    x = np.linspace(a, b, N + 1)    
    
    # Values in points.
    y = f(x)
    
    # Integral value.
    result = (h / 3) * np.sum(y[0:-1:2] + 4 * y[1::2] + y[2::2])
    
    return result    

# Error formulas 

In [5]:
def R(f, N, d, C, M, a=0, b=1):
    """Calculating error rate.
    
    Args:
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        d (int): Degree of accuracy.
        C (float): 1/2 for left rectangles.
                   1/12 for trapeezes.
                   1/2880 for simpson.
        M (float): Maximum (d+1) derivative value on [a, b].
        a, b (int, int): Bounds.
        
    Returns:
        error (float): Actual error rate.
    """
    
    # Error rate.
    error = C * (b - a) * ((b - a) / N)**(d + 1) * M
    
    return error

In [6]:
def R_main(S_N, S_2N, d):
    """Main error part.
    
    Args:
        S_N (float): cubature sum with N partitions.
        S_2N (float): cubature sum with 2N partitions.
        d (int): Degree of accuracy.
        
    Returns:
        error (float): Main error part.
    """
    
    # Error part.
    error = (S_2N - S_N) / (2**(d + 1) - 1)
    
    return error

In [7]:
def I_adjusted(S_2N, R_m):
    """Richardson extrapolation.
    
    Args:
        S_2N (float): cubature sum with 2N partitions.
        R_m (float): Main error part.        
    """
    
    I_a = S_2N + R_m
    
    return I_a

# Calculating parameters 

In [8]:
def calculate_params(method, f, N, d, C, M, a=0, b=1):
    """Calculating all params needed for table.
   
    Args:
        method: How fo we count integral. Either left_rectangles,
                trapezes, or simpson.
        f: Function that we want to count integral from.
        N (int): Number of partitions.
        d (int): Degree of accuracy.
        C (float): 1/2 for left rectangles.
                   1/12 for trapeezes.
                   1/2880 for simpson.
        M (float): Maximum (d+1) derivative value on [a, b].
        a, b (int, int): Bounds.
        
    Returns:
        params (list<float>): List of needed parameters.
    """
   
    # S_N.
    S_N = method(f=f, N=N, a=a, b=b)

    # I - S_N.
    I_S_N = I - S_N

    # R_N.
    R_N = R(f=f, N=N, d=d, C=C, M=M, a=a, b=b)

    # S_2N.
    S_2N = method(f, 2*N, a=a, b=b)

    # I - S_2N.
    I_S_2N = I - S_2N

    # R_2N.
    R_2N = R(f=f, N=2*N, d=d, C=C, M=M, a=a, b=b)

    # R_main.
    R_m = R_main(S_N, S_2N, d)

    # I_adjusted.
    I_ad = I_adjusted(S_2N, R_m)

    # I - I_adjusted.
    I_I_ad = I - I_ad

    params = np.round(np.array([S_N, I_S_N, R_N, S_2N, I_S_2N, R_2N, R_m, I_ad, I_I_ad]), 5)
    
    return params

# Calculating integrals

## Squared 

Let's define $f(x)$ as $$f(x) = x^2 + 2x + 1$$

In [9]:
def f(x):
    return x**2 + 2*x + 1

In [10]:
# Number of partitions.
N = 2

# Exact integral value.
I = integrate.quad(f, 0, 1)[0]
print("Exact integral value: {:.5f}".format(I))

# Initializing DataFrame.
columns = ["Метод", "$S_N$", "$I - S_N$", "$R_N$", "$S_{2N}$", "$I - S_{2N}$", "$R_{2N}$", "$R_{main}$", "$I_{ad}$", "$I - I_{ad}$"]
df = pd.DataFrame(columns = columns)

Exact integral value: 2.33333


### Left rectangles 

In [11]:
# Degree of accuracy.
d = 0

# Constant.
C = 1/2

# Maximum (d+1) derivative value on [a, b].
M = 4

In [12]:
left_rectangles_params = calculate_params(method=left_rectangles, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Левых прям."] + list(left_rectangles_params)

###  Trapezes

In [13]:
# Degree of accuracy.
d = 1

# Constant.
C = 1/12

# Maximum (d+1) derivative value on [a, b].
M = 2

In [14]:
trapezes_params = calculate_params(method=trapezes, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Трапеций"] + list(trapezes_params)

###  Simpson

In [15]:
# Degree of accuracy.
d = 3

# Constant.
C = 1/2880

# Maximum (d+1) derivative value on [a, b].
M = 0

In [16]:
simpson_params = calculate_params(method=simpson, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Симпсона"] + list(simpson_params)

### Result 

In [17]:
display(df.set_index("Метод"))

,$S_N$,$I - S_N$,$R_N$,$S_{2N}$,$I - S_{2N}$,$R_{2N}$,$R_{main}$,$I_{ad}$,$I - I_{ad}$
Метод,,,,,,,,,
Левых прям.,1.62500,0.70833,1.00000,1.96875,0.36458,0.50000,0.34375,2.31250,0.02083
Трапеций,2.37500,-0.04167,0.04167,2.34375,-0.01042,0.01042,-0.01042,2.33333,0.00000
Симпсона,2.33333,0.00000,0.00000,2.33333,0.00000,0.00000,0.00000,2.33333,0.00000


## Fourth power 

Let's define $f(x)$ as $$f(x) = x^4 + 8x^3 + 3x^2 + 10x + 1$$

In [18]:
def f(x):
    return x**4 + 8*x**3 + 3*x**2 + 10*x + 1

In [19]:
# Number of partitions.
N = 2

# Exact integral value.
I = integrate.quad(f, 0, 1)[0]
print("Exact integral value: {:.5f}".format(I))

# Initializing DataFrame.
columns = ["Метод", "$S_N$", "$I - S_N$", "$R_N$", "$S_{2N}$", "$I - S_{2N}$", "$R_{2N}$", "$R_{main}$", "$I_{ad}$", "$I - I_{ad}$"]
df = pd.DataFrame(columns = columns)

Exact integral value: 9.20000


### Left rectangles 

In [20]:
# Degree of accuracy.
d = 0

# Constant.
C = 1/2

# Maximum (d+1) derivative value on [a, b].
M = 44

In [21]:
left_rectangles_params = calculate_params(method=left_rectangles, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Левых прям."] + list(left_rectangles_params)

###  Trapezes

In [22]:
# Degree of accuracy.
d = 1

# Constant.
C = 1/12

# Maximum (d+1) derivative value on [a, b].
M = 66

In [23]:
trapezes_params = calculate_params(method=trapezes, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Трапеций"] + list(trapezes_params)

###  Simpson

In [24]:
# Degree of accuracy.
d = 3

# Constant.
C = 1/2880

# Maximum (d+1) derivative value on [a, b].
M = 24

In [25]:
simpson_params = calculate_params(method=simpson, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Симпсона"] + list(simpson_params)

### Result 

In [26]:
display(df.set_index("Метод"))

,$S_N$,$I - S_N$,$R_N$,$S_{2N}$,$I - S_{2N}$,$R_{2N}$,$R_{main}$,$I_{ad}$,$I - I_{ad}$
Метод,,,,,,,,,
Левых прям.,4.40625,4.79375,11.00000,6.62695,2.57305,5.50000,2.22070,8.84766,0.35234
Трапеций,9.90625,-0.70625,1.37500,9.37695,-0.17695,0.34375,-0.17643,9.20052,-0.00052
Симпсона,9.20833,-0.00833,0.00052,9.20052,-0.00052,0.00003,-0.00052,9.20000,0.00000


## Fraction

Let's define $f(x)$ as $$f(x) = \frac{1}{x^2 + 1}$$

In [27]:
def f(x):
    return 1 / (x**2 + 1)

In [28]:
# Number of partitions.
N = 2

# Exact integral value.
I = integrate.quad(f, 0, 1)[0]
print("Exact integral value: {:.5f}".format(I))

# Initializing DataFrame.
columns = ["Метод", "$S_N$", "$I - S_N$", "$R_N$", "$S_{2N}$", "$I - S_{2N}$", "$R_{2N}$", "$R_{main}$", "$I_{ad}$", "$I - I_{ad}$"]
df = pd.DataFrame(columns = columns)

Exact integral value: 0.78540


### Left rectangles 

In [29]:
# Degree of accuracy.
d = 0

# Constant.
C = 1/2

# Maximum (d+1) derivative value on [a, b].
M = 0

In [30]:
left_rectangles_params = calculate_params(method=left_rectangles, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Левых прям."] + list(left_rectangles_params)

###  Trapezes

In [31]:
# Degree of accuracy.
d = 1

# Constant.
C = 1/12

# Maximum (d+1) derivative value on [a, b].
M = 0.5

In [32]:
trapezes_params = calculate_params(method=trapezes, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Трапеций"] + list(trapezes_params)

###  Simpson

In [33]:
# Degree of accuracy.
d = 3

# Constant.
C = 1/2880

# Maximum (d+1) derivative value on [a, b].
M = 24

In [34]:
simpson_params = calculate_params(method=simpson, f=f, N=N, d=d, C=C, M=M)
df.loc[len(df)] = ["Симпсона"] + list(simpson_params)

### Result 

In [35]:
display(df.set_index("Метод"))

,$S_N$,$I - S_N$,$R_N$,$S_{2N}$,$I - S_{2N}$,$R_{2N}$,$R_{main}$,$I_{ad}$,$I - I_{ad}$
Метод,,,,,,,,,
Левых прям.,0.90000,-0.11460,0.00000,0.84529,-0.05990,0.00000,-0.05471,0.79059,-0.00519
Трапеций,0.77500,0.01040,0.01042,0.78279,0.00260,0.00260,0.00260,0.78539,0.00001
Симпсона,0.78333,0.00206,0.00052,0.78539,0.00001,0.00003,0.00014,0.78553,-0.00013
